In [1]:
from os.path import dirname, join, isfile, isdir
from datetime import datetime, timedelta
from unidecode import unidecode
from hashlib import sha256
from os import makedirs
from sys import exit
import pandas as pd
import codecs

from bulletin import __file__ as __root__
from bulletin.data.casos_confirmados import CasosConfirmados
from bulletin.data.notifica import Notifica
from bulletin.commom import static
from bulletin.commom.normalize import normalize_text, normalize_labels, normalize_number, normalize_municipios, normalize_igbe, normalize_hash, data_hash
from bulletin.commom.utils import Timer, auto_fit_columns

pd.set_option('display.max_columns', None)
output = join("output","correcoes","casos_ativos")


if not isdir(output):
    makedirs(output)

In [2]:
ativos = pd.read_excel(join("input","ativos.xlsx"), 
    converters = {
        'IBGE_RES_PR': normalize_igbe,
        'Nome': normalize_text,
        'Sexo': normalize_text,
        'Idade': lambda x: normalize_number(x,fill=0),
        'Mun Resid': normalize_text,
        'Mun atend': normalize_text,
        'RS': lambda x: normalize_number(x,fill=99),
        'Laboratório': normalize_text },
    parse_dates=False )

ativos.columns = [ normalize_labels(x) for x in ativos.columns ]
ativos = ativos.rename(columns={'ibge_res_pr': 'ibge'})

ativos = ativos.drop(index=ativos.loc[ativos['rs']==99].index, columns=['mun_resid'])
ativos['rs'] = ativos['rs'].apply(lambda x: str(x).zfill(2))

municipios = static.municipios[['ibge','municipio']].copy().rename(columns={'municipio':'mun_resid'})
municipios['ibge'] = municipios['ibge'].apply(str) 
municipios['municipio'] = municipios['ibge'].apply(normalize_text)

ativos = pd.merge(ativos,municipios,how="left",on="ibge")

ativos['ordem'] = [ x for x in range(1,len(ativos)+1) ]

ativos = ativos[['ordem','rs','ibge','mun_resid','nome','sexo','idade','mun_atend','dt_notificacao','dias_apos_notificacao']]

In [3]:
ativos = ativos.sort_values('dias_apos_notificacao',ascending=False)

In [4]:
ativos['até 20 dias'] = 0
ativos['após 20 dias'] = 0
ativos.loc[ativos['dias_apos_notificacao']<21,'até 20 dias'] = 1
ativos.loc[ativos['dias_apos_notificacao']>20,'após 20 dias'] = 1

In [5]:
writer = pd.ExcelWriter(join(output,f"ativos_PR.xlsx"),
                    engine='xlsxwriter',
                    datetime_format='dd/mm/yyyy',
                    date_format='dd/mm/yyyy')

totaisPR = ativos.groupby(['rs','ibge','mun_resid'])[['até 20 dias','após 20 dias']].sum()

totaisPR.to_excel(writer,f"Totais PR")
worksheet = writer.sheets[f"Totais PR"]
auto_fit_columns(worksheet,totaisPR)

ativos = ativos.sort_values('dias_apos_notificacao',ascending=False)
ativos[['ordem','rs','ibge','mun_resid','nome','sexo','idade','mun_atend','dt_notificacao','dias_apos_notificacao']].to_excel(writer,f"Geral")
worksheet = writer.sheets[f"Geral"]
auto_fit_columns(worksheet,ativos[['ordem','rs','ibge','mun_resid','nome','sexo','idade','mun_atend','dt_notificacao','dias_apos_notificacao']])

writer.save()

In [6]:
for rs, df_rs in ativos.groupby('rs'):
    writer = pd.ExcelWriter(join(output,f"ativos_{int(rs)}rs.xlsx"),
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')

    totais_rs = df_rs.groupby(['rs','ibge','mun_resid'])[['até 20 dias','após 20 dias']].sum()
    totais_rs.to_excel(writer,f"totais")
    worksheet = writer.sheets[f"totais"]
    auto_fit_columns(worksheet,totais_rs)
    
    df_rs = df_rs[['ordem','rs','ibge','mun_resid','nome','sexo','idade','mun_atend','dt_notificacao','dias_apos_notificacao']]
    
    df_rs.to_excel(writer,'geral',index=False)
    worksheet = writer.sheets[f"geral"]
    auto_fit_columns(worksheet,df_rs)

    for ibge, df_ibge in df_rs.groupby('ibge'):
        df_ibge.to_excel(writer,f"{ibge}",index=False)
        worksheet = writer.sheets[f"{ibge}"]
        auto_fit_columns(worksheet,df_ibge)

    writer.save()

KeyError: "Columns not found: 'após 20 dias', 'até 20 dias'"